# Finding The Words - RSS and Simple Natural Language Processing in R
##### David Miller - July 2018 - [Link to Github](https://github.com/millerdw/millerdw.github.io/tree/master/_notebooks/FindingTheWords_1)
---

Changing tack a little since my last post in [Agricultural Approaches](), I wanted to skip ahead to something I'm particularly interested in. This is a big segue from my previous posts (not segway, that's something else entirely), hence the start of a new blog series. This way, I'm hoping to swap backwards and forwards between simple and higher level concepts, and between different subjects without bamboozling myself, or indeed anyone else, in the process.

I'm really interested in the ways in which algorithms can be used to analyse text. There is a profound amount of content, both generated historically and produced daily, that is based in language. Words are the closest humans have to representations of distinct ideas, and it's absolutely incredible how we use them.

At the risk of falling into a recursive loop; consider what I've written so far on the page, and how the black and white scratchings have (hopefully) conveyed meaning well above and beyond the pixels that make them up. Consider further how figurative language like "scratchings" and "above and beyond" don't just provide a literal meaning, but tap into associations that are common between a writer and their audience. Consider even further how when I refer to "a writer" and "their audience", even without any further information it's clear that I'm referring to myself and all of you because that's how people with an admittedly flowery disposition with words have spoken to you in the past... Consider even further again how... gaahhh abort,abort,abort...

Language is incredibly powerful and is a perfect insight into how humans not only perceive, but understand, the world around them. Beyond this, it has been shown that *Language affects a human's understanding of the world*; there is a feedback effect, whereby as language is learnt, our vocabulary *affects* how we differentiate between real world stimuli. This is known as the [Sapir-Whorf Hypothesis](http://www.linguisticsnetwork.com/wp-content/uploads/What-is-the-Sapir_Whorf-Hypothesis.compressed.pdf), and it might just be my favourite of all the 'ideas' or theories that I've ever come across (tied closely with the supposed etymology of the word "foreign" but, as this is probably apocryphal, I'll leave it for another time).

It's for these reasons I believe that understanding language is the key to understanding how we think. In this blog I want to explore the ways in which an algorithm can begin to get a handle on the former, and the steps required for us to begin to approximate the latter. 

I'll start with pretty simple ideas, because I'm learning here too, but hopefully we'll progress into areas that are genuinely taxing and interesting. Anyway, I've rambled on enough, I hope you enjoy it.

### Spit it Out

I always find that it's best to start with a good example, or project, in mind. In this case, I've been interested in the disparity between reportage of different news outlets for some time, even more so since the tail has begun to wag the dog in terrifying ways (see the EU Referendum and the 2016 US Election). I don't mean that as a political point, it's more that I remember the shock of finding out, twice in a year, that what I'd thought was a sure bet, what everything I'd read had said was a sure thing, turned out to be anything but. I decided that in an ideal world, whatever a persons views were, whatever their voting intention, everyone should be presented with the same facts, without bias or agenda. In reality this is near impossible, but the next best thing, surely, is to take all of those different biased and agenda driven publications, on both sides of the argument, and play them off against each other.

The trouble is, this data doesn't come nicely labelled for us. There are often tags telling us roughly what topic an article relates to (e.g. World, Sport, Lifestyle), but not describing the specific story. So, for the course of this series, my aim will be to **group different articles together based solely on their content**. 

I'm going to try and get stuck in as soon as possible and work things out as I go along, as is my habit, so hold on if the narrative takes a leap of faith or two, but hopefully you'll be able to see a structure emerge as we make some progress.


### Straight from the horse's mouth
## Accesssing RSS feeds

First things first, I'm going to walk through **collecting data from RSS feeds using an R library**. 

I'm using R in this case because it was the language that I originally began to prototype my model in when I started a few months back, and frankly it seems like a waste of time translating it into something else. I like to think I'm a clean coder though, so regardless of you're own personal preference of the tools to use, you should be able to follow along.

[RSS](https://en.wikipedia.org/wiki/RSS) (has lots of bacronymic translations, but I know it as Really Simple Syndication) feeds are just streams of text-based content organised in something akin to an XML format. It contains markers to denote the start and end of objects within the text stream, and has a limited set of properties for each object (Date, Title, Author, and so on). RSS appears to be falling out of favour lately, being replaced mainly by [json](https://www.json.org/) (JavaScript Object Notation), which allows for more flexibility in the objects being passed and parsed between the source server and the consumer application. More on that at another time, I guess.

Because of this common structure in RSS, it is *very* easy to parse the text stream into lists of objects in a programming environment, as a result there are lots of libraries available to help you do this, especially in an very open community like R's. Here I'm using a library called [feedeR](https://cran.r-project.org/web/packages/feedeR/index.html) to do the dirty work for me

**[Note]** A word of warning, always remember that the library your downloading is written by someone else who may or may not be a better, more diligent, or more motivated developer than you. This may not matter a jot (as in this case), but if you're considering building applications off of one then remember that you'll have absolutely noone to blame but yourself if a nasty bug emerges from said library at a later point in time. Remember also that each library will have a set of it's own weird dependencies, so you can download one or two and in reality be relying on five or ten different sets of code. (Being a .NET developer originally, I try to avoid building full blown apps in R anyway - there are better tools for that so use them - that said, very little beats this language in terms of prototyping and ad hoc analysis).

Below I've gone through the process of installing and loading all of the libraries relevant to our cause:


In [2]:
install.packages(c("stringdist","feedeR","foreach","doParallel","rvest","magrittr"))

library(doParallel)
registerDoParallel(makeCluster(4))

library(foreach)
library(magrittr)

library(feedeR)

Installing packages into 'C:/Users/David/Documents/R/win-library/3.3'
(as 'lib' is unspecified)
Warning message:
"packages 'feedeR', 'foreach', 'doParallel' are in use and will not be installed"

package 'stringdist' successfully unpacked and MD5 sums checked
package 'rvest' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\David\AppData\Local\Temp\Rtmp0SzSox\downloaded_packages


Yes, forgive me, I've set up a library to add a little parallelism-juice to the process. This works simply with the [foreach](https://cran.r-project.org/web/packages/foreach/index.html) library. I've also included [magrittr](https://cran.r-project.org/web/packages/magrittr/index.html) to make R more like FSharp...

Now for the more interesting part. Below I've listed a series of active RSS sites I was able to find through google (hopefully of  different political persuasions), and I'm

In [15]:

## GATHER RAW DATA

# Define links for feeds
feeds <- c("http://feeds.bbci.co.uk/news/world/rss.xml",
            "http://feeds.bbci.co.uk/news/rss.xml",
            "http://feeds.skynews.com/feeds/rss/uk.xml",
            "http://feeds.skynews.com/feeds/rss/world.xml",
            "http://feeds.skynews.com/feeds/rss/us.xml",
            "http://feeds.reuters.com/Reuters/domesticNews",
            "http://feeds.reuters.com/Reuters/worldNews",
            "http://feeds.foxnews.com/foxnews/national",
            "http://feeds.foxnews.com/foxnews/world",
            "http://rssfeeds.usatoday.com/UsatodaycomWorld-TopStories",
            "http://rssfeeds.usatoday.com/UsatodaycomNation-TopStories",
            "http://rss.nytimes.com/services/xml/rss/nyt/World.xml",
            "http://www.nytimes.com/services/xml/rss/nyt/Africa.xml",
            "http://www.nytimes.com/services/xml/rss/nyt/Americas.xml",
            "http://www.nytimes.com/services/xml/rss/nyt/AsiaPacific.xml",
            "http://www.nytimes.com/services/xml/rss/nyt/Europe.xml",
            "http://www.nytimes.com/services/xml/rss/nyt/MiddleEast.xml",
            "http://www.nytimes.com/services/xml/rss/nyt/US.xml",
            "http://www.telegraph.co.uk/news/rss.xml"
            )


#download the dataset
dataset <- 
    #iterate through each feed address
    foreach(i=1:length(feeds), .combine=rbind) %do% {
        tryCatch({
            # extract items from each feed
            extract <- feed.extract(feeds[i])
            # return items with additional field of parent feed        
            cbind.data.frame(feed = extract$title, extract$items)
         }, error = function(e) { NULL }) #return NULL on an error (i.e. ignore it)
    }

#remove any duplicate entries, by checking each items url
dataset <- dataset[match(unique(dataset$link), dataset$link),]

#titles <- sapply(dataset$title, function(s) {(strsplit(gsub("[[:punct:]]", " ", tolower(s)), " ")) })

#show first 10
dataset[1:10,]

Space required after the Public Identifier
SystemLiteral " or ' expected
SYSTEM or PUBLIC, the URI is missing
Space required after the Public Identifier
SystemLiteral " or ' expected
SYSTEM or PUBLIC, the URI is missing
Space required after the Public Identifier
SystemLiteral " or ' expected
SYSTEM or PUBLIC, the URI is missing
Space required after the Public Identifier
SystemLiteral " or ' expected
SYSTEM or PUBLIC, the URI is missing
Space required after the Public Identifier
SystemLiteral " or ' expected
SYSTEM or PUBLIC, the URI is missing
Space required after the Public Identifier
SystemLiteral " or ' expected
SYSTEM or PUBLIC, the URI is missing


feed,title,date,link,hash
BBC News - World,Trump-Putin summit: US president reverses remark on Russia meddling,2018-07-17 20:52:59,https://www.bbc.co.uk/news/world-us-canada-44864739,0bc39dc7f5641a1d
BBC News - World,Barack Obama condemns disregard for facts,2018-07-17 18:31:16,https://www.bbc.co.uk/news/world-africa-44858937,ff2a4b417287dd7e
BBC News - World,Alabama employee gets new car from boss after 20-mile walk,2018-07-17 16:20:18,https://www.bbc.co.uk/news/world-us-canada-44854370,a40f669e2a0ad253
BBC News - World,ECHR condemns Pussy Riot and Anna Politkovskaya cases,2018-07-17 12:08:10,https://www.bbc.co.uk/news/world-europe-44857461,b8cb4b10cc9746d4
BBC News - World,"Trump to redesign Air Force One to be 'red, white and blue'",2018-07-17 18:36:31,https://www.bbc.co.uk/news/world-us-canada-44865953,21587f160eb6ccd5
BBC News - World,Spain sexual consent: PM Pedro Sanchez promises new law,2018-07-17 17:35:31,https://www.bbc.co.uk/news/world-europe-44866759,e2ee037064703bb6
BBC News - World,Kim Jong-un blasts delays in North Korean economic projects,2018-07-17 13:04:13,https://www.bbc.co.uk/news/world-asia-44855699,6a3481bfd42e59de
BBC News - World,Israel suspends fuel deliveries to Gaza over arson attacks,2018-07-17 11:06:33,https://www.bbc.co.uk/news/world-middle-east-44858637,f69cd308f2c350c2
BBC News - World,"Las Vegas shooting: Mandalay Bay hotel owner sues 1,000 victims",2018-07-17 13:58:10,https://www.bbc.co.uk/news/world-us-canada-44859238,5aae1479a20a9962
BBC News - World,Houston police arrest 'mattress murders' suspect,2018-07-17 15:53:23,https://www.bbc.co.uk/news/world-us-canada-44854368,2c0aa4ddc22d3c30


Excellent! There you have the top 10 stories from the feeds we investigated. 

Believe it or not, that's a pretty big step.

In [16]:
nrow(dataset)

[1] 290

In [7]:

#feed contents
#library(rvest)

contents <- foreach(i = 1:nrow(dataset), .combine = rbind, .packages = "rvest") %dopar% {
    #i <- 10
    tryCatch({
        page <- read_html(dataset$link[i])
        paragraphs <- html_nodes(page, "p")
        p_classes <- html_attr(paragraphs, "class")
        p_text <- html_text(paragraphs)
        words <- as.character(unlist(sapply(p_text[is.na(p_classes) | grepl("intro", tolower(p_classes))],
                                            function(s) { strsplit(s, " ")})))
        out <- sapply(words[1:100], function(s) { tolower(gsub("[^[:alnum:][:space:]]", "", s)) })
        names(out) <- NULL
        #c(hash = dataset$hash[i], out)
        c(id=i, out)
    }, error = function(e) { "" })
}
#contents[1,]


In [8]:
nrow(contents)

[1] 272

In [9]:

## VECTORISE DATA

#create wordvector
contentWords <- table(c(contents))
contentWords <- cbind.data.frame(word = tolower(names(contentWords)[1:length(names(contentWords))]),
                                 count = contentWords)

wordVector <- unique(c(tolower(unlist(titles)),
                       tolower(unlist(contents))))
wordVector <- wordVector[wordVector != ""]


vectorisedData <- foreach(i = 1:nrow(dataset), .combine=rbind) %dopar% {
    #i <- 1
    tVector <- integer(length(wordVector))
    row <- contents[contents[, "id"] == i,]
    countVector <- table(c(titles[[i]], row[2:length(row)]))
    for (j in 1:length(countVector)) {
        index <- match(names(countVector)[j], wordVector)
        tVector[index] <- countVector[j]
    }
    t(tVector)
    #vectorisedData <- rbind.data.frame(vectorisedData, t(as.data.frame(tVector))) #tVector
}
names(vectorisedData) <- wordVector

vectorisedData <- as.matrix(vectorisedData)
colnames(vectorisedData) <- wordVector


In [10]:

## Word Bundles
correlationmatrix <- cor(vectorisedData)
bundles <- as.data.frame(foreach(i = 1:ncol(vectorisedData), .combine = rbind) %do% {
    #i<-1
    vec <- correlationmatrix[, i]
    bundle <- vec[vec > 0.9]
    c(id = i,
      bundle = paste(names(bundle[!is.na(bundle)]), collapse = "-"),
      value = sum(bundle[!is.na(bundle)]),
      first = colnames(correlationmatrix)[i],
      length = length(bundle[!is.na(bundle)]))
})
bundles <- bundles[match(unique(bundles$bundle[as.numeric(bundles$value) > 1]), bundles$bundle),]



Warning message in cor(vectorisedData):
"the standard deviation is zero"

In [11]:

## Synonyms
library(stringdist)
similaritymatrix <- matrix(foreach(i = 1:length(wordVector), .combine = rbind, .packages = "stringdist") %dopar% { stringdist(wordVector[i], wordVector) },
                           ncol = length(wordVector),
                           nrow = length(wordVector),
                           dimnames = list(wordVector,wordVector))
synonyms <- as.data.frame(foreach(i = 1:ncol(vectorisedData), .combine = rbind) %do% {
    #i<-1
    vec <- similaritymatrix[, i]
    synonym <- vec[vec < 0.25 * length(colnames(similaritymatrix)[i])]
    c(id = i,
      synonym = paste(names(synonym[!is.na(synonym)]), collapse = "-"),
      value = sum(synonym[!is.na(synonym)]))
})
synonyms <- synonyms[match(unique(synonyms$synonym[as.numeric(synonyms$value) > 1]), synonyms$synonym),]


In [12]:

## SELECT FEATURES
analytics <- cbind.data.frame(wordVector,
                              count = sapply(1:length(wordVector), function(w) { sum(vectorisedData[, w]) }),
                              mean = sapply(1:length(wordVector), function(w) { mean(vectorisedData[, w]) }),
                              stdev = sapply(1:length(wordVector), function(w) { sd(vectorisedData[, w]) }),
                              max = sapply(1:length(wordVector), function(w) { max(vectorisedData[, w]) }),
                              min = sapply(1:length(wordVector), function(w) { min(vectorisedData[, w]) }))

analytics$varration <- sapply(1:nrow(analytics), function(a) { analytics$stdev[a] / analytics$mean[a] })

test <- top_n(analytics, 50, analytics$count)
test <- analytics[rank(analytics$count, ties.method = "random"),]

lengths <- sapply(1:nrow(vectorisedData), function(i) { sum(vectorisedData[i,]) })
lengths <- cbind.data.frame(length = lengths)


library(dplyr)
#analytics[sort(analytics$count, decreasing = T),]
#selectedFeatures <- as.character(analytics[!is.na(analytics$varration) & analytics$varration > 2,]$wordVector)
#selectedFeatures <- as.character(top_n(analytics, 200, analytics$varration)$wordVector)
#selectedFeatures <- names(vectorisedData)
selectedFeatures <- as.character(bundles$first[as.numeric(bundles$length)>2])

featureSet <- matrix(as.numeric(vectorisedData[, selectedFeatures]),nrow = nrow(vectorisedData))#,
                               #date = with(dataset, (as.numeric(date) - quantile(as.numeric(date), 0.05)) / (mean(as.numeric(date)) - quantile(as.numeric(date), 0.05))),
                               #hash_ = dataset$hash)



ERROR: Error in eval(expr, envir, enclos): could not find function "top_n"


In [ ]:

## K MEANS CLUSTERING
cartesianDistance <- function(v1, v2) {
    #sum(mapply(function(c1, c2) {(c1 - c2) ^ 2 }, v1, v2))^0.5
    ((v1 - v2) %*% t(v1 - v2))[[1]]^0.5
}

generateCentroid <- function() {
    centroid <- double(length(selectedFeatures))
    #centroid[sample(1:length(selectedFeatures), round(mean(lengths$length) + 1))] <- 1
    centroid <- as.numeric(featureSet[sample(1:nrow(dataset),1),])
    centroid
}

k <- 10
centroids <- list()
for (i in 1:k) {
    centroids <- rbind.data.frame(centroids, generateCentroid())
}
centroids <- as.matrix(centroids)
colnames(centroids)<- NULL  

G <- 100
history <- list()
g <- 1
while (g <= G) {
    dataset$cluster <- foreach(i = 1:nrow(featureSet), .combine = rbind) %dopar% {
        #i <- 1
        distances <- sapply(1:k, function(c) {
            #c <- 2
            V1 <- as.matrix(centroids[c,])
            V2 <- as.matrix(featureSet[i, ])
            (t(V1-V2) %*% (V1-V2))[[1]]^0.5
        })
        match(min(distances), distances)
    }

    newcentroids <- centroids
    for (i in 1:k) {
        #i <- 2
        if (nrow(dataset[dataset$cluster == i, ]) > 1) {
            #i <- 1
            members <- as.data.frame(featureSet[dataset$cluster == i,])
            for (j in 1:length(selectedFeatures)) {
                newcentroids[i, j] = mean(members[, j])
            }
        } else {
            newcentroid <- generateCentroid()
            for (j in 1:length(selectedFeatures)) {
                newcentroids[i, j] = newcentroid[j]
            }
        }
    }
    lastCentroids <- as.matrix(centroids)
    centroids <- as.matrix(newcentroids)

    dataset$distance <- foreach(i = 1:nrow(featureSet), .combine = rbind) %dopar% {
        # i <- 1
        V1 <- as.matrix(centroids[dataset$cluster[i],]) #, nrow = 1)
        V2 <- as.matrix(featureSet[i,])
        (t(V1 - V2) %*% (V1 - V2))[[1]] ^ 0.5
    }
                                     
    distances <- double(k)
    for (i in 1:k) {
        distances[i] <- sum(dataset$distance[dataset$cluster == i]) / length(dataset$distance[dataset$cluster == i])
    }
    history <- rbind.data.frame(history, cbind(g, t(distances), sum(distances)))

    g <- g + 1
}
names(history) <- c("generation", 1:k, "total")
View(history)


In [ ]:

## ANALYSIS

test<-dataset[dataset$cluster==1,]
View(test)

test <- dataset[order(dataset$cluster, dataset$distance),]
View(test)

test2 <- merge(x = featureSet[, names(featureSet)[!names(featureSet) %in% selectedFeatures]],
               y = dataset,
               by.x = "hash_",
               by.y = "hash")

history

cl <- 9
result <- top_n(test2[test2$clusters_ == cl,], 10, test2$distance_[test2$clusters_ == cl])
View(result)
